# Age at creation for programming languages stats

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[MathematicaForPrediction at WordPress](https://mathematicaforprediction.wordpress.com)   
May 2024



## Introduction

In this notebook we ingest programming languages creation data from ["](https://pldb.io/index.html)**[P](https://pldb.io/index.html)**[rogramming ](https://pldb.io/index.html)**[L](https://pldb.io/index.html)**[anguage ](https://pldb.io/index.html)**[D](https://pldb.io/index.html)**[ata](https://pldb.io/index.html)**[B](https://pldb.io/index.html)**[ase"](https://pldb.io/index.html) and visualize several statistics of it.

We do not examine the data source and we do not want to reason too much about the data using the stats. We started this notebook by just wanting to make the bubble charts (both 2D and 3D.) Nevertheless, we are tempted to say and justify statements like:

- Pareto holds, as usual.

- Language creators tend to do it more than once.

- Beware the [Second system effect](https://en.wikipedia.org/wiki/Second-system_effect).

### References

Here are reference links with explanations and links to dataset files:

- [The Ages of Programming Language Creators (pldb.io)](https://pldb.io/posts/ageAtCreation.html)

    - Short note about data and related statistics; provides a link to a [TSV file](https://pldb.io/posts/age.tsv) with the data.

- [The Ages of Programming Language Creators (datawrapper.dwcdn.net)](https://datawrapper.dwcdn.net/rT0yG/1/)

    - "Just a plot"; provides a link to a CSV file with the data.

- [The Ages of Programming Language Creators](https://www.reddit.com/r/programming/comments/1cw2ri4/the_ages_of_programming_language_creators/) (Reddit)

    - Link(s) and discussion.

------

## Setup

In [1]:
use Data::Importers;
use Data::Reshapers;
use Data::Summarizers;

use JavaScript::D3;

In [2]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [3]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none')


## Data ingestion

Here we get the TSC file with Wolfram Function Repository (WFR) function [ImportCSVToDataset](https://resources.wolframcloud.com/FunctionRepository/resources/ImportCSVToDataset/):

In [4]:
my $url = "https://pldb.io/posts/age.tsv";
my @dsDataLines = data-import($url).lines.map({ $_.split("\t") })>>.Array;
deduce-type(@dsDataLines)

Vector(Vector(Atom((Str)), 13), 216)

In [5]:
my @field-names = @dsDataLines.head.Array;
my @dsData = @dsDataLines.tail(*-2).map({ @field-names.Array Z=> $_.Array })>>.Hash;

deduce-type(@dsData)

Vector(Assoc(Atom((Str)), Atom((Str)), 13), 214)

In [6]:
@dsData = @dsData.map({
    $_<ageAtCreation> = $_<ageAtCreation>.UInt;
    $_<rank> = $_<rank>.Int;
    $_<pldbScore> = $_<pldbScore>.Int;
    $_<appeared> = $_<appeared>.Int;
    $_<numberOfUsersEstimate> = $_<numberOfUsersEstimate>.Int;
    $_<numberOfJobsEstimate> = $_<numberOfJobsEstimate>.Int;
    $_<foundationScore> = $_<foundationScore>.Int;
    $_
}).Array;

deduce-type(@dsData)

Vector(Struct([ageAtCreation, appeared, creators, foundationScore, id, inboundLinksCount, measurements, name, numberOfJobsEstimate, numberOfUsersEstimate, pldbScore, rank, tags], [Int, Int, Str, Int, Str, Str, Str, Str, Int, Int, Int, Int, Str]), 214)

Show summary:

In [7]:
sink records-summary(@dsData, max-tallies => 12, field-names => @field-names.sort[^7]);

+---------------------+-----------------------+-------------------------------+---------------------+--------------------+-------------------+----------------+
| ageAtCreation       | appeared              | creators                      | foundationScore     | id                 | inboundLinksCount | measurements   |
+---------------------+-----------------------+-------------------------------+---------------------+--------------------+-------------------+----------------+
| Min    => 16        | Min    => 1948        | Niklaus Wirth          => 8   | Min    => 0         | mathematica => 1   | 0       => 102    | 11      => 12  |
| 1st-Qu => 30        | 1st-Qu => 1978        | Breck Yunits           => 7   | 1st-Qu => 0         | gcc         => 1   | 1       => 32     | 15      => 12  |
| Mean   => 36.766355 | Mean   => 1993.009346 | John Backus            => 5   | Mean   => 30.303738 | perl-6      => 1   | 2       => 9      | 16      => 10  |
| Median => 35        | Median => 1994.5

In [8]:
sink records-summary(@dsData, max-tallies => 12, field-names => @field-names.sort[7..12])

+----------------------------+-----------------------+-------------------------+------------------------+---------------------+---------------------------+
| name                       | numberOfJobsEstimate  | numberOfUsersEstimate   | pldbScore              | rank                | tags                      |
+----------------------------+-----------------------+-------------------------+------------------------+---------------------+---------------------------+
| Python              => 1   | Min    => 0           | Min    => 0             | Min    => 15930        | Min    => 2         | pl                 => 135 |
| scribble            => 1   | 1st-Qu => 0           | 1st-Qu => 176           | 1st-Qu => 20916        | 1st-Qu => 80        | textMarkup         => 9   |
| CSS                 => 1   | Mean   => 7963.630841 | Mean   => 240720.794393 | Mean   => 22145.481308 | Mean   => 844.85514 | dataNotation       => 9   |
| Bel                 => 1   | Median => 0           | Median =>

Focus languages to be used in the plots below:

In [9]:
my @focusLangs = ["C++", "Fortran", "Java", "Mathematica", "Perl 6", "Raku", "SQL", "Wolfram Language"];

[C++ Fortran Java Mathematica Perl 6 Raku SQL Wolfram Language]

Here we find the most important tags (used in the plots below):

In [10]:
my @topTags = @dsData.map(*<tags>).&tally.sort({ $_.value }).reverse.head(7)>>.key;

[pl dataNotation textMarkup library grammarLanguage stylesheetLanguage queryLanguage]

Here we add the column "group" based on the focus languages and most important tags:

In [11]:
@dsData = @dsData.map({ 
    $_<group> = do if $_<name> ∈ @focusLangs { "focus" } elsif $_<tags> ∈ @topTags { $_<tags> } else { "other" };
    $_
});

deduce-type(@dsData)

Vector(Struct([ageAtCreation, appeared, creators, foundationScore, group, id, inboundLinksCount, measurements, name, numberOfJobsEstimate, numberOfUsersEstimate, pldbScore, rank, tags], [Int, Int, Str, Int, Str, Str, Str, Str, Str, Int, Int, Int, Int, Str]), 214)

------

## Distributions

Here are the distributions of the variables/columns:

- age at creation 

    - i.e. "How old was the creator?"

- appeared"

    - i.e. "In what year the programming language was proclaimed?"

In [12]:
#% js
my %opts = title-color => 'Silver', background => 'none', bins => 40;
js-d3-histogram(@dsData.map(*<ageAtCreation>), title => 'Age at creation', |%opts) 
~
js-d3-histogram(@dsData.map(*<appeared>), title => 'Appeared', |%opts)


Here are corresponding Box-Whisker plots:

In [13]:
select-columns(@dsData, <name ageAtCreation>).raku

[{:ageAtCreation(35), :name("Python")}, {:ageAtCreation(31), :name("C")}, {:ageAtCreation(40), :name("Java")}, {:ageAtCreation(35), :name("C++")}, {:ageAtCreation(36), :name("HTML")}, {:ageAtCreation(31), :name("CSS")}, {:ageAtCreation(30), :name("Ruby")}, {:ageAtCreation(33), :name("Perl")}, {:ageAtCreation(27), :name("PHP")}, {:ageAtCreation(53), :name("Go")}, {:ageAtCreation(41), :name("XML")}, {:ageAtCreation(46), :name("JSON")}, {:ageAtCreation(52), :name("TypeScript")}, {:ageAtCreation(30), :name("SQL")}, {:ageAtCreation(40), :name("C#")}, {:ageAtCreation(39), :name("R")}, {:ageAtCreation(30), :name("Bash")}, {:ageAtCreation(44), :name("PowerShell")}, {:ageAtCreation(36), :name("Swift")}, {:ageAtCreation(33), :name("Rust")}, {:ageAtCreation(33), :name("Lua")}, {:ageAtCreation(46), :name("Scala")}, {:ageAtCreation(45), :name("MATLAB")}, {:ageAtCreation(38), :name("Haskell")}, {:ageAtCreation(27), :name("Kotlin")}, {:ageAtCreation(48), :name("Clojure")}, {:ageAtCreation(31), :name(

In [15]:
#% js
my %opts = :horizontal, :outliers, title-color => 'Silver', stroke-color => 'White', background => 'none', width => 400;
js-d3-box-whisker-chart(@dsData.map(*<ageAtCreation>), title => 'Age at creation', |%opts)
~
js-d3-box-whisker-chart(@dsData.map(*<appeared>), title => 'Appeared', |%opts)

Here are tables of the corresponding statistics:

In [16]:
my @field-names = <ageAtCreation appeared>;
sink records-summary(select-columns(@dsData, @field-names), :@field-names)

+---------------------+-----------------------+
| ageAtCreation       | appeared              |
+---------------------+-----------------------+
| Min    => 16        | Min    => 1948        |
| 1st-Qu => 30        | 1st-Qu => 1978        |
| Mean   => 36.766355 | Mean   => 1993.009346 |
| Median => 35        | Median => 1994.5      |
| 3rd-Qu => 42        | 3rd-Qu => 2008        |
| Max    => 70        | Max    => 2023        |
+---------------------+-----------------------+



## Pareto principle manifestation

### Number of creations

Here is the Pareto principle statistic for the number of created (or renamed) programming languages per creator:

In [17]:
my %creations = @dsData.map(*<creators>).&tally;
my @paretoStats = pareto-principle-statistic(%creations);
@paretoStats.head(6)

(Niklaus Wirth => 0.037383 Breck Yunits => 0.070093 John Backus => 0.093458 Tim Berners-Lee => 0.11215 Chris Lattner => 0.130841 Larry Wall => 0.149533)

Here is the corresponding plot:

In [18]:
#% js
js-d3-list-plot( @paretoStats>>.value, 
    title => 'Pareto principle: number languages per creators team', 
    title-color => 'Silver',
    background => 'none', 
    :grid-lines,
)

**Remark:** We can see that ≈25% of the creators correspond to ≈50% of the languages.


### Popularity

Obviously, programmers can and do use more than one programming language. Nevertheless, it is interesting to see the Pareto principle plot for the languages "mind share" based on the number of users *estimates*.

In [19]:
#% js
my %users = @dsData.map({ $_<id> => $_<numberOfUsersEstimate>.Int });
my @paretoStats = pareto-principle-statistic(%users);

js-d3-list-plot( @paretoStats>>.value, 
    title => 'Pareto principle: number users per language', 
    title-color => 'Silver',
    background => 'none', 
    :grid-lines,
)

**Remark:** Again, the plot above is "wrong" -- programmers use more than one programming language.


-----------

## Correlations

In order to see meaningful correlation, pairwise plots we take logarithms of the large value columns:

In [20]:
my @corColnames = <appeared ageAtCreation numberOfUsersEstimate numberOfJobsEstimate rank measurements>;
my @dsDataVar = select-columns(@dsData, @corColnames);
@dsDataVar = @dsDataVar.map({ 
    my %h = $_.clone; 
    %h<numberOfUsersEstimate> = log(%h<numberOfUsersEstimate> + 1, 10); 
    %h<numberOfJobsEstimate> = log(%h<numberOfJobsEstimate> + 1, 10);
    %h
}).Array;

deduce-type(@dsDataVar)


Vector(Struct([ageAtCreation, appeared, measurements, numberOfJobsEstimate, numberOfUsersEstimate, rank], [Int, Int, Str, Num, Num, Int]), 214)

Here make a Cartesian product of the focus columns and make scatter points plot for each pair of that product:

In [21]:
#% js
(@corColnames X @corColnames)>>.reverse>>.Array.map( -> $c {
    my @points = @dsDataVar.map({ %( x => $_{$c.head}, y => $_{$c.tail} ) });
    js-d3-list-plot( @points, width => 180, height => 180, x-label => $c.head, y-label => $c.tail )
}).join("\n")

**Remark:** Given the names of the data columns and the corresponding obvious interpretations we can say that the stronger correlations make sense.

--------

## Bubble chart 2D

In this section we make an informative 2D bubble chart with (tooltips).

Here we make a dataset for the bubble chart:

In [22]:
my @dsData2 = @dsData.map({
    %( x => $_<appeared>, y => $_<ageAtCreation>, z => log($_<numberOfUsersEstimate>, 10), group => $_<group>, label => "<b>{$_<name>}</b> by {$_<creators>}")
});

deduce-type(@dsData2)

Vector(Struct([group, label, x, y, z], [Str, Str, Int, Int, Num]), 214)

Here is the bubble chart:

In [23]:
#% js
js-d3-bubble-chart(@dsData2, 
        z-range-min => 1,
        z-range-max => 16,
        title-color => 'Silver',
        title-font-size => 20,
        x-label => "appeared", 
        y-label => "lg(rank)",
        title => 'Age at creation',
        width => 1200,
        margins => { left => 60, bottom => 50, right => 200},
        background => 'none',
        :grid-lines
);

**Remark:** The programming language J is a clear outlier because of creators' ages.

-------

## Second system effect traces


In this section we try -- and fail -- to demonstrate that the more programming languages a team of creators makes the less successful those languages are. (Maybe, because they are more cumbersome and suffer the Second system effect?)

**Remark:** This section is mostly made "for fun." It is not true that each sets of languages per creators team is made of comparable languages. For example, complementary languages can be in the same set. (See, HTTP, HTML, URL.) Some sets are just made of the same language but with different names. (See, Perl 6 and Raku, and Mathematica and Wolfram Language.) Also, older languages would have the [First mover advantage](https://en.wikipedia.org/wiki/First-mover_advantage).

Make creators to index association:

In [24]:
my %creators = @dsData.map(*<creators>).&tally.pairs.grep(*.value > 1);
my %nameToIndex = %creators.keys.sort Z=> ^%creators.elems;
%nameToIndex.elems

40

Make a bubble chart dataset with relative popularity per creators team:

In [25]:
my @nUsers = @dsData.grep({ %creators{$_<creators>}:exists });

@nUsers = |group-by(@nUsers, <creators>).map({ 

    my $m = max(1, $_.value.map(*<numberOfUsersEstimate>).max.sqrt);

    $_.value.map({ %( x => $_<appeared>, y => %nameToIndex{$_<creators>}, z => $_<numberOfUsersEstimate>.sqrt/$m, group => $_<creators>, label => "<b>{$_<name>}</b>" ) }) 
    
})>>.Array.flat;

@nUsers .= sort(*<group>);

deduce-type(@nUsers)

Vector(Struct([group, label, x, y, z], [Str, Str, Int, Int, Num]), 110)

Here is the corresponding bubble chart:

In [26]:
#% js
js-d3-bubble-chart(@nUsers, 
        z-range-min => 1,
        z-range-max => 16,
        title => 'Second system effect',
        title-color => 'Silver',
        title-font-size => 20,
        x-label => "appeared",
        y-label => "creators", 
        z-range-min => 3,
        z-range-max => 10,
        width => 1000,
        height => 900,
        margins => { left => 60, bottom => 50, right => 200},
        background => 'none',
        grid-lines => (Whatever, %nameToIndex.elems),
        opacity => 0.9,
);

From the plot above we *cannot* decisively say that:

> The most recent creation of a team of programming language creators is not team's most popular creation.

That statement, though, does hold for a fair amount of cases.


-------

## References

TBD...